In [1]:
import pandas as pd
import numpy as np

Загрузка данных

In [2]:
train = pd.read_csv("data/train.csv")

questions = pd.read_csv('data/questions.csv')

lectures = pd.read_csv('data/lectures.csv')

Просмотр первых строк каждого набора данных

In [6]:
train.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [7]:
questions.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [8]:
lectures.head()

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


Проверка информации о наборах данных

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int64  
 3   content_id                      int64  
 4   content_type_id                 int64  
 5   task_container_id               int64  
 6   user_answer                     int64  
 7   answered_correctly              int64  
 8   prior_question_elapsed_time     float64
 9   prior_question_had_explanation  object 
dtypes: float64(1), int64(8), object(1)
memory usage: 7.5+ GB


In [10]:
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13523 non-null  int64 
 1   bundle_id       13523 non-null  int64 
 2   correct_answer  13523 non-null  int64 
 3   part            13523 non-null  int64 
 4   tags            13522 non-null  object
dtypes: int64(4), object(1)
memory usage: 528.4+ KB


In [11]:
lectures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lecture_id  418 non-null    int64 
 1   tag         418 non-null    int64 
 2   part        418 non-null    int64 
 3   type_of     418 non-null    object
dtypes: int64(3), object(1)
memory usage: 13.2+ KB


Проверка наличия пропущенных значений

In [12]:
train.isnull().sum()

row_id                                  0
timestamp                               0
user_id                                 0
content_id                              0
content_type_id                         0
task_container_id                       0
user_answer                             0
answered_correctly                      0
prior_question_elapsed_time       2351538
prior_question_had_explanation     392506
dtype: int64

In [13]:
questions.isnull().sum()

question_id       0
bundle_id         0
correct_answer    0
part              0
tags              1
dtype: int64

In [14]:
lectures.isnull().sum()

lecture_id    0
tag           0
part          0
type_of       0
dtype: int64

Убедимся, что столбцы имеют правильные типы данных. Если необходимо, преобразуем их.

In [15]:
train.dtypes

row_id                              int64
timestamp                           int64
user_id                             int64
content_id                          int64
content_type_id                     int64
task_container_id                   int64
user_answer                         int64
answered_correctly                  int64
prior_question_elapsed_time       float64
prior_question_had_explanation     object
dtype: object

In [17]:
# Разделение данных на вопросы и лекции
train_questions = train[train['content_type_id'] == 0].copy()
train_lectures = train[train['content_type_id'] == 1].copy()

In [18]:
# Объединение train_questions с questions.csv по content_id и question_id
train_questions = train_questions.merge(questions, how='left', left_on='content_id', right_on='question_id')

Распределение правильных и неправильных ответов

In [22]:
# Подсчет количества правильных и неправильных ответов
answer_counts = train_questions['answered_correctly'].value_counts()

print(answer_counts)

answered_correctly
1    65244627
0    34026673
Name: count, dtype: int64


Вывод:
Большинство ответов студентов являются правильными, что может свидетельствовать о достаточном уровне подготовки или эффективности обучающего материала.

Влияние времени, затраченного на предыдущие вопросы, на успеваемость

In [23]:
# Создание групп по времени, затраченному на предыдущие вопросы
train_questions['elapsed_time_bins'] = pd.cut(train_questions['prior_question_elapsed_time'], bins=10)

# Расчет среднего процента правильных ответов для каждой группы времени
elapsed_time_accuracy = train_questions.groupby('elapsed_time_bins')['answered_correctly'].mean()

print("\nСредний процент правильных ответов в зависимости от времени на предыдущий вопрос:")
print(elapsed_time_accuracy)

/var/folders/1s/5zrmnc61429fjndvw10l0mb80000gn/T/ipykernel_53161/259696149.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elapsed_time_accuracy = train_questions.groupby('elapsed_time_bins')['answered_correctly'].mean()



Средний процент правильных ответов в зависимости от времени на предыдущий вопрос:
elapsed_time_bins
(-300.0, 30000.0]       0.662263
(30000.0, 60000.0]      0.640120
(60000.0, 90000.0]      0.649248
(90000.0, 120000.0]     0.637229
(120000.0, 150000.0]    0.627435
(150000.0, 180000.0]    0.621926
(180000.0, 210000.0]    0.619436
(210000.0, 240000.0]    0.618372
(240000.0, 270000.0]    0.621057
(270000.0, 300000.0]    0.618686
Name: answered_correctly, dtype: float64


Интерпретация:
Общая тенденция: с увеличением времени, затраченного на предыдущие вопросы, средний процент правильных ответов снижается.
Наименьшее время (до 30 000 мс): самый высокий средний процент правильных ответов (66,23%).
Наибольшее время (от 270 000 мс): самый низкий средний процент правильных ответов (61,87%).
Вывод:
Быстрые ответы на предыдущие вопросы могут свидетельствовать о уверенности и хорошей подготовке студентов, что положительно сказывается на последующей успеваемости.
Длительное время на предыдущие вопросы может указывать на затруднения, усталость или потерю концентрации, что негативно влияет на успех в следующих заданиях.

Влияние наличия объяснения на предыдущий вопрос на успеваемость

In [24]:
# Расчет среднего процента правильных ответов в зависимости от prior_question_had_explanation
explanation_accuracy = train_questions.groupby('prior_question_had_explanation')['answered_correctly'].mean()

print("\nСредний процент правильных ответов в зависимости от наличия объяснения к предыдущему вопросу:")
print(explanation_accuracy)


Средний процент правильных ответов в зависимости от наличия объяснения к предыдущему вопросу:
prior_question_had_explanation
False    0.507977
True     0.673189
Name: answered_correctly, dtype: float64


Интерпретация:
Студенты, получившие объяснение к предыдущему вопросу, имеют средний процент правильных ответов 67,32%.
Студенты без объяснения показывают средний результат 50,80%.
Вывод:
Наличие объяснения после ответа на вопрос значительно повышает успеваемость студентов в последующих заданиях.
Рекомендация: интеграция объяснений и разборов после вопросов может улучшить общую успеваемость.

Влияние части теста (part) на успеваемость

In [25]:
# Расчет среднего процента правильных ответов по каждой части теста
part_accuracy = train_questions.groupby('part')['answered_correctly'].mean().sort_values(ascending=False)

print("\nСредний процент правильных ответов по частям теста:")
print(part_accuracy)


Средний процент правильных ответов по частям теста:
part
1    0.745032
2    0.708694
3    0.701456
6    0.669388
7    0.659601
4    0.630998
5    0.610088
Name: answered_correctly, dtype: float64


Интерпретация:
Часть 1: самый высокий средний процент правильных ответов (74,50%).
Часть 5: самый низкий средний процент правильных ответов (61,01%).
Разница между частями: успеваемость варьируется от части к части.
Вывод:
Сложность разделов: некоторые части теста являются более сложными для студентов.


Влияние тега вопроса (tags) на успеваемость

In [26]:
# Выделение первого тега
train_questions['first_tag'] = train_questions['tags'].apply(lambda x: int(str(x).split(' ')[0]) if pd.notnull(x) else np.nan)

# Расчет среднего процента правильных ответов по первым тегам
tag_accuracy = train_questions.groupby('first_tag')['answered_correctly'].mean().sort_values(ascending=False)

print("\nСредний процент правильных ответов по первым тегам (топ 10):")
print(tag_accuracy.head(10))


Средний процент правильных ответов по первым тегам (топ 10):
first_tag
38.0     0.944345
41.0     0.806974
177.0    0.801152
51.0     0.796269
131.0    0.795531
115.0    0.789888
32.0     0.769989
129.0    0.747204
137.0    0.738926
62.0     0.738633
Name: answered_correctly, dtype: float64


Интерпретация:
Тег 38.0: самый высокий средний процент правильных ответов (94,43%).
Теги с высоким процентом правильных ответов могут указывать на темы, хорошо освоенные студентами или на более простые вопросы.
Вывод:
Определенные темы (теги) являются более понятными для студентов.


Анализ опыта пользователя (количество ранее отвеченных вопросов)

In [27]:
# Сортировка данных по user_id и timestamp
train_questions.sort_values(['user_id', 'timestamp'], inplace=True)

# Добавление счетчика ранее отвеченных вопросов для каждого пользователя
train_questions['user_question_count'] = train_questions.groupby('user_id').cumcount() + 1  # +1, чтобы счет начинался с 1

# Создание бинов по количеству отвеченных вопросов
train_questions['question_count_bins'] = pd.cut(train_questions['user_question_count'], bins=10)

# Расчет среднего процента правильных ответов в зависимости от опыта
experience_accuracy = train_questions.groupby('question_count_bins')['answered_correctly'].mean()

print("\nСредний процент правильных ответов в зависимости от опыта пользователя:")
print(experience_accuracy)

/var/folders/1s/5zrmnc61429fjndvw10l0mb80000gn/T/ipykernel_53161/2201660114.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  experience_accuracy = train_questions.groupby('question_count_bins')['answered_correctly'].mean()



Средний процент правильных ответов в зависимости от опыта пользователя:
question_count_bins
(-16.608, 1761.8]     0.644470
(1761.8, 3522.6]      0.696169
(3522.6, 5283.4]      0.709188
(5283.4, 7044.2]      0.720542
(7044.2, 8805.0]      0.725327
(8805.0, 10565.8]     0.733045
(10565.8, 12326.6]    0.752598
(12326.6, 14087.4]    0.770685
(14087.4, 15848.2]    0.799089
(15848.2, 17609.0]    0.777380
Name: answered_correctly, dtype: float64


Интерпретация:
Рост опыта: с увеличением количества ранее отвеченных вопросов средний процент правильных ответов увеличивается.
Начальный уровень (до ~1761 вопросов): средний процент правильных ответов 64,45%.
Высокий уровень опыта (более ~14 000 вопросов): средний процент правильных ответов достигает до 79,91%.
Вывод:
Опыт положительно влияет на успеваемость студентов.
